In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('nba_games.csv')
df = df.sort_values('date')
df = df.reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df

,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,+/-_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,team,total,home,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,23.0,36.0,59.0,23.0,5.0,3.0,15.0,15.0,106.0,0.493,0.448,0.302,0.271,41.1,83.7,59.6,62.2,5.2,5.5,12.3,111.2,98.6,7.0,19.0,0.600,4.0,7.0,0.571,6.0,10.0,1.0,8.0,11.0,19.0,5.0,2.0,2.0,3.0,4.0,21.0,26.0,0.685,0.643,0.833,0.625,18.5,41.2,24.8,35.6,3.2,4.7,33.3,23.6,132.0,104.0,DET,106,0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,7.0,33.0,40.0,22.0,9.0,4.0,15.0,25.0,94.0,0.530,0.500,0.329,0.183,16.3,58.9,40.4,59.5,9.4,6.0,14.5,98.6,111.2,8.0,16.0,1.000,2.0,6.0,1.000,3.0,4.0,1.000,3.0,7.0,8.0,4.0,4.0,3.0,5.0,4.0,20.0,10.0,1.330,1.500,1.000,2.000,20.5,28.8,17.9,34.6,10.6,7.1,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,7.0,33.0,40.0,22.0,9.0,4.0,15.0,25.0,94.0,0.530,0.500,0.329,0.183,16.3,58.9,40.4,59.5,9.4,6.0,14.5,98.6,111.2,8.0,16.0,1.000,2.0,6.0,1.000,3.0,4.0,1.0,3.0,7.0,8.0,4.0,4.0,3.0,5.0,4.0,20.0,10.0,1.330,1.500,1.000,2.000,20.5,28.8,17.9,34.6,10.6,7.1,57.1,33.8,258.0,121.0,ATL,94,1,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,23.0,36.0,59.0,23.0,5.0,3.0,15.0,15.0,106.0,0.493,0.448,0.302,0.271,41.1,83.7,59.6,62.2,5.2,5.5,12.3,111.2,98.6,7.0,19.0,0.600,4.0,7.0,0.571,6.0,10.0,1.000,8.0,11.0,19.0,5.0,2.0,2.0,3.0,4.0,21.0,26.0,0.685,0.643,0.833,0.625,18.5,41.2,24.8,35.6,3.2,4.7,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,39.0,50.0,26.0,5.0,7.0,10.0,21.0,95.0,0.468,0.452,0.309,0.181,21.6,84.8,51.5,68.4,5.0,10.3,9.0,95.5,97.5,12.0,22.0,0.571,3.0,7.0,0.500,3.0,4.0,1.0,4.0,10.0,12.0,7.0,1.0,2.0,3.0,4.0,25.0,9.0,0.714,0.714,1.000,2.000,10.8,41.9,23.8,31.2,2.8,18.5,30.4,29.0,138.0,105.0,CLE,95,0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,40.0,47.0,13.0,6.0,10.0,13.0,22.0,97.0,0.499,0.466,0.218,0.264,15.2,78.4,48.5,35.1,6.0,15.4,11.8,97.5,95.5,8.0,22.0,0.750,3.0,5.0,0.750,5.0,5.0,1.000,2.0,8.0,10.0,5.0,2.0,6.0,4.0,6.0,19.0,9.0,0.820,0.875,0.556,1.333,12.2,38.5,26.0,30.3,2.8,14.0,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,40.0,47.0,13.0,6.0,10.0,13.0,22.0,97.0,0.499,0.466,0.218,0.264,15.2,78.4,48.5,35.1,6.0,15.4,11.8,97.5,95.5,8.0,22.0,0.750,3.0,5.0,0.750,5.0,5.0,1.0,2.0,8.0,10.0,5.0,2.0,6.0,4.0,6.0,19.0,9.0,0.820,0.875,0.556,1.333,12.2,38.5,26.0,30.3,2.8,14.0,53.2,34.6,162.0,104.0,CHI,97,1,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,39.0,50.0,26.0,5.0,7.0,10.0,21.0,95.0,0.468,0.452,0.309,0.181,21.6,84.8,51.5,68.4,5.0,10.3,9.0,95.5,97.5,12.0,22.0,0.571,3.0,7.0,0.500,3.0,4.0,1.000,4.0,10.0,12.0,7.0,1.0,2.0,3.0,4.0,25.0,9.0,0.714,0.714,1.000,2.000,10.8,41.9,23.8,31.2,2.8,18.5,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,8.0,25.0,33.0,21.0,9.0,3.0,18.0,26.0,95.0,0.501,0.458,0.217,0.325,18.6,54.3,37.1,60.0,9.0,4.5,15.9,94.9,110.9,7.0,20

In [3]:
def add_target(team):
    
    team['target'] = team['won'].shift(-1)
    
    return team

df = df.groupby('team', group_keys=False).apply(add_target)

In [4]:
df['target'][pd.isnull(df['target'])] = 2
df['target'] = df['target'].astype(int, errors='ignore')
df

D:\Anaconda\envs\mlev\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,+/-_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,team,total,home,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,23.0,36.0,59.0,23.0,5.0,3.0,15.0,15.0,106.0,0.493,0.448,0.302,0.271,41.1,83.7,59.6,62.2,5.2,5.5,12.3,111.2,98.6,7.0,19.0,0.600,4.0,7.0,0.571,6.0,10.0,1.0,8.0,11.0,19.0,5.0,2.0,2.0,3.0,4.0,21.0,26.0,0.685,0.643,0.833,0.625,18.5,41.2,24.8,35.6,3.2,4.7,33.3,23.6,132.0,104.0,DET,106,0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,7.0,33.0,40.0,22.0,9.0,4.0,15.0,25.0,94.0,0.530,0.500,0.329,0.183,16.3,58.9,40.4,59.5,9.4,6.0,14.5,98.6,111.2,8.0,16.0,1.000,2.0,6.0,1.000,3.0,4.0,1.000,3.0,7.0,8.0,4.0,4.0,3.0,5.0,4.0,20.0,10.0,1.330,1.500,1.000,2.000,20.5,28.8,17.9,34.6,10.6,7.1,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True,1
1,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,7.0,33.0,40.0,22.0,9.0,4.0,15.0,25.0,94.0,0.530,0.500,0.329,0.183,16.3,58.9,40.4,59.5,9.4,6.0,14.5,98.6,111.2,8.0,16.0,1.000,2.0,6.0,1.000,3.0,4.0,1.0,3.0,7.0,8.0,4.0,4.0,3.0,5.0,4.0,20.0,10.0,1.330,1.500,1.000,2.000,20.5,28.8,17.9,34.6,10.6,7.1,57.1,33.8,258.0,121.0,ATL,94,1,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,23.0,36.0,59.0,23.0,5.0,3.0,15.0,15.0,106.0,0.493,0.448,0.302,0.271,41.1,83.7,59.6,62.2,5.2,5.5,12.3,111.2,98.6,7.0,19.0,0.600,4.0,7.0,0.571,6.0,10.0,1.000,8.0,11.0,19.0,5.0,2.0,2.0,3.0,4.0,21.0,26.0,0.685,0.643,0.833,0.625,18.5,41.2,24.8,35.6,3.2,4.7,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False,1
2,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,39.0,50.0,26.0,5.0,7.0,10.0,21.0,95.0,0.468,0.452,0.309,0.181,21.6,84.8,51.5,68.4,5.0,10.3,9.0,95.5,97.5,12.0,22.0,0.571,3.0,7.0,0.500,3.0,4.0,1.0,4.0,10.0,12.0,7.0,1.0,2.0,3.0,4.0,25.0,9.0,0.714,0.714,1.000,2.000,10.8,41.9,23.8,31.2,2.8,18.5,30.4,29.0,138.0,105.0,CLE,95,0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,40.0,47.0,13.0,6.0,10.0,13.0,22.0,97.0,0.499,0.466,0.218,0.264,15.2,78.4,48.5,35.1,6.0,15.4,11.8,97.5,95.5,8.0,22.0,0.750,3.0,5.0,0.750,5.0,5.0,1.000,2.0,8.0,10.0,5.0,2.0,6.0,4.0,6.0,19.0,9.0,0.820,0.875,0.556,1.333,12.2,38.5,26.0,30.3,2.8,14.0,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False,1
3,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,40.0,47.0,13.0,6.0,10.0,13.0,22.0,97.0,0.499,0.466,0.218,0.264,15.2,78.4,48.5,35.1,6.0,15.4,11.8,97.5,95.5,8.0,22.0,0.750,3.0,5.0,0.750,5.0,5.0,1.0,2.0,8.0,10.0,5.0,2.0,6.0,4.0,6.0,19.0,9.0,0.820,0.875,0.556,1.333,12.2,38.5,26.0,30.3,2.8,14.0,53.2,34.6,162.0,104.0,CHI,97,1,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,39.0,50.0,26.0,5.0,7.0,10.0,21.0,95.0,0.468,0.452,0.309,0.181,21.6,84.8,51.5,68.4,5.0,10.3,9.0,95.5,97.5,12.0,22.0,0.571,3.0,7.0,0.500,3.0,4.0,1.000,4.0,10.0,12.0,7.0,1.0,2.0,3.0,4.0,25.0,9.0,0.714,0.714,1.000,2.000,10.8,41.9,23.8,31.2,2.8,18.5,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True,1
4,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,8.0,25.0,33.0,21.0,9.0,3.0,18.0,26.0,95.0,0.501,0.458,0.217,0.325,18.6,54.3,37.1,60.0,9.0,4.5,15.9,94

In [5]:
df['won'].value_counts()

True     10004
False    10004
Name: won, dtype: int64

In [40]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier
from xgboost import XGBRegressor

rr = RidgeClassifier(alpha=1)
xgb = XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1)
split = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(rr, n_features_to_select=40, direction='forward', cv=split)

In [7]:
removed_columns = ['season', 'date', 'won', 'target', 'team', 'team_opp']

In [8]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [10]:
df

,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,+/-_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,team,total,home,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.391304,0.529412,0.277512,0.413793,0.378788,0.491686,0.441860,0.396825,0.730455,0.793103,0.454545,0.725490,0.372093,0.227273,0.15,0.482759,0.250000,0.375000,0.342380,0.318271,0.397218,0.326196,0.736559,0.707885,0.759459,0.586552,0.239631,0.136139,0.421642,0.472009,0.333699,0.190476,0.261905,0.384615,0.285714,0.260870,0.571,0.20,0.257143,1.0,0.615385,0.40,0.576923,0.130435,0.2,0.181818,0.181818,0.50,0.180328,0.586667,0.185,0.208680,0.79125,0.040049,0.185,0.304965,0.175439,0.261468,0.032,0.047,0.300839,0.020513,0.203791,0.317647,DET,0.375000,0.0,0.391304,0.323529,0.435407,0.275862,0.348485,0.351544,0.255814,0.222222,0.766628,0.241379,0.386364,0.352941,0.348837,0.409091,0.20,0.482759,0.527778,0.267857,0.419624,0.420432,0.438949,0.215365,0.292115,0.263441,0.240541,0.547926,0.433180,0.148515,0.503731,0.333699,0.472009,0.238095,0.190476,1.000000,0.142857,0.217391,1.000,0.08,0.085714,1.000000,0.230769,0.20,0.153846,0.086957,0.4,0.272727,0.363636,0.50,0.163934,0.373333,0.830,1.000000,1.00000,0.138735,0.205,0.158392,0.099781,0.250000,0.106,0.071,0.550314,0.151282,0.800948,0.517647,ATL,0.267857,1.0,2016,2015-10-27,True,1
1,0.391304,0.323529,0.435407,0.275862,0.348485,0.351544,0.255814,0.222222,0.766628,0.241379,0.386364,0.352941,0.348837,0.409091,0.20,0.482759,0.527778,0.267857,0.419624,0.420432,0.438949,0.215365,0.292115,0.263441,0.240541,0.547926,0.433180,0.148515,0.503731,0.333699,0.472009,0.238095,0.190476,1.000000,0.142857,0.217391,1.000,0.08,0.085714,1.0,0.230769,0.20,0.153846,0.086957,0.4,0.272727,0.363636,0.50,0.163934,0.373333,0.830,1.000000,1.00000,0.138735,0.205,0.158392,0.099781,0.250000,0.106,0.071,0.550314,0.151282,0.800948,0.517647,ATL,0.267857,1.0,0.391304,0.529412,0.277512,0.413793,0.378788,0.491686,0.441860,0.396825,0.730455,0.793103,0.454545,0.725490,0.372093,0.227273,0.15,0.482759,0.250000,0.375000,0.342380,0.318271,0.397218,0.326196,0.736559,0.707885,0.759459,0.586552,0.239631,0.136139,0.421642,0.472009,0.333699,0.190476,0.261905,0.384615,0.285714,0.260870,0.571,0.20,0.257143,1.000000,0.615385,0.40,0.576923,0.130435,0.2,0.181818,0.181818,0.50,0.180328,0.586667,0.185,0.208680,0.79125,0.040049,0.185,0.304965,0.175439,0.261468,0.032,0.047,0.300839,0.020513,0.203791,0.317647,DET,0.375000,0.0,2016,2015-10-27,False,1
2,0.413043,0.500000,0.322967,0.310345,0.378788,0.368171,0.209302,0.253968,0.519253,0.379310,0.522727,0.549020,0.441860,0.227273,0.35,0.310345,0.416667,0.276786,0.290188,0.326130,0.408037,0.212846,0.387097,0.727599,0.540541,0.675250,0.230415,0.254950,0.298507,0.299671,0.321625,0.428571,0.333333,0.340000,0.214286,0.260870,0.500,0.08,0.085714,1.0,0.307692,0.35,0.307692,0.217391,0.1,0.181818,0.181818,0.50,0.245902,0.360000,0.214,0.274238,1.00000,0.138735,0.108,0.313239,0.164474,0.211009,0.028,0.185,0.270440,0.089744,0.232227,0.329412,CLE,0.276786,0.0,0.391304,0.397059,0.373206,0.241379,0.227273,0.43705

In [11]:
def backtest(data, model, predictors, start=2, step=1):
    
    all_predictions = []
    
    seasons = sorted(data['season'].unique())
    
    for i in range(start, len(seasons), step):
        
        season = seasons[i]
        
        train = data[data['season'] < season]
        test = data[data['season'] == season]
        
        model.fit(train[predictors], train['target'])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds,index=test.index)
        
        combined = pd.concat([test['target'], preds], axis=1)
        combined.columns = ['actual', 'prediction']
        
        all_predictions.append(combined)
        
    return pd.concat(all_predictions)

In [12]:
def summary(data):
    
    try:
        summary_df = data[['team', 'team_opp', 'date']]
    except:
        summary_df = data[['team_x', 'team_opp', 'date']]
        
    return summary_df

In [13]:
df_rolling = df[list(selected_columns) + ['won', 'team', 'season']]

In [14]:
def find_team_averages(team):
    
    rolling = team.rolling(10).mean()
    
    return rolling


df_rolling = df_rolling.groupby(['team', 'season'], group_keys=False).apply(find_team_averages)    

In [15]:
df_rolling

,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,+/-_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,total,home,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,won,season
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [16]:
rolling_cols = [f'{col}_10' for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis=1)

In [17]:
df = df.dropna()
df

,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,+/-_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,team,total,home,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target,fg_10,fga_10,fg%_10,3p_10,3pa_10,3p%_10,ft_10,fta_10,ft%_10,orb_10,drb_10,trb_10,ast_10,stl_10,blk_10,tov_10,pf_10,pts_10,ts%_10,efg%_10,3par_10,ftr_10,orb%_10,drb%_10,trb%_10,ast%_10,stl%_10,blk%_10,tov%_10,ortg_10,drtg_10,fg_max_10,fga_max_10,fg%_max_10,3p_max_10,3pa_max_10,3p%_max_10,ft_max_10,fta_max_10,ft%_max_10,orb_max_10,drb_max_10,trb_max_10,ast_max_10,stl_max_10,blk_max_10,tov_max_10,pf_max_10,pts_max_10,+/-_max_10,ts%_max_10,efg%_max_10,3par_max_10,ftr_max_10,orb%_max_10,drb%_max_10,trb%_max_10,ast%_max_10,stl%_max_10,blk%_max_10,tov%_max_10,usg%_max_10,ortg_max_10,drtg_max_10,total_10,home_10,fg_opp_10,fga_opp_10,fg%_opp_10,3p_opp_10,3pa_opp_10,3p%_opp_10,ft_opp_10,fta_opp_10,ft%_opp_10,orb_opp_10,drb_opp_10,trb_opp_10,ast_opp_10,stl_opp_10,blk_opp_10,tov_opp_10,pf_opp_10,pts_opp_10,ts%_opp_10,efg%_opp_10,3par_opp_10,ftr_opp_10,orb%_opp_10,drb%_opp_10,trb%_opp_10,ast%_opp_10,stl%_opp_10,blk%_opp_10,tov%_opp_10,ortg_opp_10,drtg_opp_10,fg_max_opp_10,fga_max_opp_10,fg%_max_opp_10,3p_max_opp_10,3pa_max_opp_10,3p%_max_opp_10,ft_max_opp_10,fta_max_opp_10,ft%_max_opp_10,orb_max_opp_10,drb_max_opp_10,trb_max_opp_10,ast_max_opp_10,stl_max_opp_10,blk_max_opp_10,tov_max_opp_10,pf_max_opp_10,pts_max_opp_10,+/-_max_opp_10,ts%_max_opp_10,efg%_max_opp_10,3par_max_opp_10,ftr_max_opp_10,orb%_max_opp_10,drb%_max_opp_10,trb%_max_opp_10,ast%_max_opp_10,stl%_max_opp_10,blk%_max_opp_10,tov%_max_opp_10,usg%_max_opp_10,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,won_10,season_10
243,0.500000,0.382353,0.523923,0.344828,0.333333,0.457245,0.255814,0.238095,0.708285,0.275862,0.454545,0.431373,0.441860,0.454545,0.35,0.379310,0.277778,0.375000,0.503132,0.512770,0.397218,0.219144,0.358423,0.673835,0.564865,0.582260,0.474654,0.321782,0.388060,0.451153,0.360044,0.380952,0.190476,0.692308,0.285714,0.217391,1.000,0.24,0.200000,1.0,0.230769,0.55,0.461538,0.347826,0.3,0.272727,0.181818,0.50,0.262295,0.426667,0.367,0.415512,1.00000,0.090863,0.120,0.369976,0.209430,0.431193,0.045,0.116,0.176101,0.087179,0.360190,0.400000,ATL,0.375000,1.0,0.391304,0.338235,0.423445,0.482759,0.378788,0.573634,0.209302,0.222222,0.611435,0.275862,0.363636,0.352941,0.325581,0.272727,0.00,0.620690,0.416667,0.303571,0.455115,0.479371,0.469861,0.212846,0.326165,0.641577,0.435135,0.509299,0.285714,0.000000,0.615672,0.360044,0.451153,0.333333,0.285714,1.000000,0.285714,0.260870,0.667,0.12,0.142857,1.000000,0.384615,0.20,0.269231,0.347826,0.2,0.000000,0.181818,0.75,0.262295,0.266667,0.402,0.538319,0.54500,0.425824,0.156,0.308511,0.191886,0.378440,0.041,0.000,0.425577,0.137179,0.161137,0.435294,NOP,0.303571,0.0,2016,2015-11-11,True,0,0.447826,0.357353,0.482775,0.320690,0.328788,0.430641,0.346512,0.295238,0.792649,0.279310,0.379545,0.368627,0.460465,0.490909,0.260,0.455172,0.327778,0.360714,0.487265,0.471513,0.400464,0.282746,0.343369,0.534588,0.432973,0.661946,

In [18]:
def shift_col(team, col_name):
    
    next_col = team[col_name].shift(-1)
    
    return next_col

def add_col(df, col_name):
    
    return df.groupby('team', group_keys=False).apply(lambda x: shift_col(x, col_name))

df['home_next'] = add_col(df, 'home')
df['team_opp_next'] = add_col(df, 'team_opp')
df['date_next'] = add_col(df, 'date')

In [30]:
df = df.copy()
next_games = ['MIL', 'DET', 'DAL', 'IND', 'HOU', 'NYK', 'PHX', 'UTA', 'PHI', 'DEN', 'NOP', 'POR', 'MIN', 'SAC', 'CHI', 'LAC']
dates = ['2023-03-23', '2023-03-24', '2023-03-25', '2023-03-26']

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

dates = pd.to_datetime(dates)
df_predict = df[df['date'].isin(dates)]
df_predict = df_predict[df_predict['team'].isin(next_games)]

df_predict = df_predict[df_predict['date_next'].isna()]
df_predict

,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,+/-_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,team,total,home,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target,fg_10,fga_10,fg%_10,3p_10,3pa_10,3p%_10,ft_10,fta_10,ft%_10,orb_10,drb_10,trb_10,ast_10,stl_10,blk_10,tov_10,pf_10,pts_10,ts%_10,efg%_10,3par_10,ftr_10,orb%_10,drb%_10,trb%_10,ast%_10,stl%_10,blk%_10,tov%_10,ortg_10,drtg_10,fg_max_10,fga_max_10,fg%_max_10,3p_max_10,3pa_max_10,3p%_max_10,ft_max_10,fta_max_10,ft%_max_10,orb_max_10,drb_max_10,trb_max_10,ast_max_10,stl_max_10,blk_max_10,tov_max_10,pf_max_10,pts_max_10,+/-_max_10,ts%_max_10,efg%_max_10,3par_max_10,ftr_max_10,orb%_max_10,drb%_max_10,trb%_max_10,ast%_max_10,stl%_max_10,blk%_max_10,tov%_max_10,usg%_max_10,ortg_max_10,drtg_max_10,total_10,home_10,fg_opp_10,fga_opp_10,fg%_opp_10,3p_opp_10,3pa_opp_10,3p%_opp_10,ft_opp_10,fta_opp_10,ft%_opp_10,orb_opp_10,drb_opp_10,trb_opp_10,ast_opp_10,stl_opp_10,blk_opp_10,tov_opp_10,pf_opp_10,pts_opp_10,ts%_opp_10,efg%_opp_10,3par_opp_10,ftr_opp_10,orb%_opp_10,drb%_opp_10,trb%_opp_10,ast%_opp_10,stl%_opp_10,blk%_opp_10,tov%_opp_10,ortg_opp_10,drtg_opp_10,fg_max_opp_10,fga_max_opp_10,fg%_max_opp_10,3p_max_opp_10,3pa_max_opp_10,3p%_max_opp_10,ft_max_opp_10,fta_max_opp_10,ft%_max_opp_10,orb_max_opp_10,drb_max_opp_10,trb_max_opp_10,ast_max_opp_10,stl_max_opp_10,blk_max_opp_10,tov_max_opp_10,pf_max_opp_10,pts_max_opp_10,+/-_max_opp_10,ts%_max_opp_10,efg%_max_opp_10,3par_max_opp_10,ftr_max_opp_10,orb%_max_opp_10,drb%_max_opp_10,trb%_max_opp_10,ast%_max_opp_10,stl%_max_opp_10,blk%_max_opp_10,tov%_max_opp_10,usg%_max_opp_10,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,won_10,season_10,home_next,team_opp_next,date_next
19951,0.391304,0.411765,0.361244,0.413793,0.545455,0.356295,0.441860,0.396825,0.730455,0.448276,0.340909,0.431373,0.441860,0.363636,0.40,0.413793,0.250000,0.375000,0.425887,0.398821,0.633694,0.356423,0.485663,0.632616,0.516216,0.702432,0.377880,0.358911,0.395522,0.440176,0.496158,0.333333,0.285714,1.000000,0.357143,0.391304,0.500,0.32,0.257143,0.833333,0.461538,0.30,0.307692,0.217391,0.3,0.272727,0.363636,0.50,0.245902,0.506667,0.500,0.538319,1.00000,0.037393,0.196,0.185579,0.140351,0.223624,0.079,0.110,0.650943,0.107692,0.582938,0.517647,NYK,0.375000,0.0,0.456522,0.397059,0.456938,0.448276,0.424242,0.482185,0.395349,0.317460,0.833139,0.275862,0.431818,0.411765,0.418605,0.363636,0.35,0.448276,0.361111,0.419643,0.517745,0.487230,0.499227,0.288413,0.367384,0.514337,0.483784,0.590844,0.377880,0.361386,0.436567,0.496158,0.440176,0.190476,0.214286,1.000000,0.285714,0.260870,0.667,0.24,0.200000,1.000000,0.153846,0.20,0.153846,0.260870,0.2,0.181818,0.181818,0.50,0.180328,0.453333,0.500,0.538319,1.00000,0.090863,0.143,0.160757,0.149123,0.227064,0.036,0.081,0.203354,0.123077,0.597156,0.435294,ORL,0.419643,1.0,2023,2023-03-23,False,2,0.480435,0.414706,0.476316,0.424138,0.506061,0.395962,0.474419,0.422222,0.744107,0.444828,0.402273,0.482353,0.369767,0.318182,0.215,0.410345,0.377778,0.463393,0.522756,0.495481,0.584235,0.3

In [33]:
def add_next_game_info(df):
    # Create empty lists for the new columns
    home_next = []
    team_opp_next = []
    date_next = []
    
    # Loop over the rows of the DataFrame
    for index, row in df.iterrows():
        # Display the 'team' column and ask for the 'home_next' value
        print('Team:', row['team'])
        home_next_input = input('Home Next: ')
        home_next.append(home_next_input)

        # Ask for the 'team_opp_next' value
        team_opp_next_input = input('Team Opponent Next: ')
        team_opp_next.append(team_opp_next_input)

    # Add the new columns to the DataFrame
    df['home_next'] = home_next
    df['team_opp_next'] = team_opp_next
    df['date_next'] = '2023-03-27'

    # Return the updated DataFrame
    return df

add_next_game_info(df_predict)

Team: NYK


Home Next:  1.0
Team Opponent Next:  HOU


Team: DET


Home Next:  1.0
Team Opponent Next:  MIL


Team: SAC


...

Team: HOU


Home Next:  0.0
Team Opponent Next:  NYK


Team: DAL


Home Next:  0.0
Team Opponent Next:  IND


,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,+/-_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,team,total,home,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target,fg_10,fga_10,fg%_10,3p_10,3pa_10,3p%_10,ft_10,fta_10,ft%_10,orb_10,drb_10,trb_10,ast_10,stl_10,blk_10,tov_10,pf_10,pts_10,ts%_10,efg%_10,3par_10,ftr_10,orb%_10,drb%_10,trb%_10,ast%_10,stl%_10,blk%_10,tov%_10,ortg_10,drtg_10,fg_max_10,fga_max_10,fg%_max_10,3p_max_10,3pa_max_10,3p%_max_10,ft_max_10,fta_max_10,ft%_max_10,orb_max_10,drb_max_10,trb_max_10,ast_max_10,stl_max_10,blk_max_10,tov_max_10,pf_max_10,pts_max_10,+/-_max_10,ts%_max_10,efg%_max_10,3par_max_10,ftr_max_10,orb%_max_10,drb%_max_10,trb%_max_10,ast%_max_10,stl%_max_10,blk%_max_10,tov%_max_10,usg%_max_10,ortg_max_10,drtg_max_10,total_10,home_10,fg_opp_10,fga_opp_10,fg%_opp_10,3p_opp_10,3pa_opp_10,3p%_opp_10,ft_opp_10,fta_opp_10,ft%_opp_10,orb_opp_10,drb_opp_10,trb_opp_10,ast_opp_10,stl_opp_10,blk_opp_10,tov_opp_10,pf_opp_10,pts_opp_10,ts%_opp_10,efg%_opp_10,3par_opp_10,ftr_opp_10,orb%_opp_10,drb%_opp_10,trb%_opp_10,ast%_opp_10,stl%_opp_10,blk%_opp_10,tov%_opp_10,ortg_opp_10,drtg_opp_10,fg_max_opp_10,fga_max_opp_10,fg%_max_opp_10,3p_max_opp_10,3pa_max_opp_10,3p%_max_opp_10,ft_max_opp_10,fta_max_opp_10,ft%_max_opp_10,orb_max_opp_10,drb_max_opp_10,trb_max_opp_10,ast_max_opp_10,stl_max_opp_10,blk_max_opp_10,tov_max_opp_10,pf_max_opp_10,pts_max_opp_10,+/-_max_opp_10,ts%_max_opp_10,efg%_max_opp_10,3par_max_opp_10,ftr_max_opp_10,orb%_max_opp_10,drb%_max_opp_10,trb%_max_opp_10,ast%_max_opp_10,stl%_max_opp_10,blk%_max_opp_10,tov%_max_opp_10,usg%_max_opp_10,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,won_10,season_10,home_next,team_opp_next,date_next
19951,0.391304,0.411765,0.361244,0.413793,0.545455,0.356295,0.441860,0.396825,0.730455,0.448276,0.340909,0.431373,0.441860,0.363636,0.40,0.413793,0.250000,0.375000,0.425887,0.398821,0.633694,0.356423,0.485663,0.632616,0.516216,0.702432,0.377880,0.358911,0.395522,0.440176,0.496158,0.333333,0.285714,1.000000,0.357143,0.391304,0.500,0.32,0.257143,0.833333,0.461538,0.30,0.307692,0.217391,0.3,0.272727,0.363636,0.50,0.245902,0.506667,0.500,0.538319,1.00000,0.037393,0.196,0.185579,0.140351,0.223624,0.079,0.110,0.650943,0.107692,0.582938,0.517647,NYK,0.375000,0.0,0.456522,0.397059,0.456938,0.448276,0.424242,0.482185,0.395349,0.317460,0.833139,0.275862,0.431818,0.411765,0.418605,0.363636,0.35,0.448276,0.361111,0.419643,0.517745,0.487230,0.499227,0.288413,0.367384,0.514337,0.483784,0.590844,0.377880,0.361386,0.436567,0.496158,0.440176,0.190476,0.214286,1.000000,0.285714,0.260870,0.667,0.24,0.200000,1.000000,0.153846,0.20,0.153846,0.260870,0.2,0.181818,0.181818,0.50,0.180328,0.453333,0.500,0.538319,1.00000,0.090863,0.143,0.160757,0.149123,0.227064,0.036,0.081,0.203354,0.123077,0.597156,0.435294,ORL,0.419643,1.0,2023,2023-03-23,False,2,0.480435,0.414706,0.476316,0.424138,0.506061,0.395962,0.474419,0.422222,0.744107,0.444828,0.402273,0.482353,0.369767,0.318182,0.215,0.410345,0.377778,0.463393,0.522756,0.495481,0.584235,0.3

In [34]:
df = pd.concat([df, df_predict], axis=0)

In [35]:
full = df.merge(
    df[rolling_cols + ['team_opp_next', 'date_next', 'team']],
    left_on=['team', 'date_next'], 
    right_on=['team_opp_next', 'date_next']
)

In [36]:
full[full['date_next'] == '2023-03-27']

,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,+/-_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,team_x,total,home,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target,fg_10_x,fga_10_x,fg%_10_x,3p_10_x,3pa_10_x,3p%_10_x,ft_10_x,fta_10_x,ft%_10_x,orb_10_x,drb_10_x,trb_10_x,ast_10_x,stl_10_x,blk_10_x,tov_10_x,pf_10_x,pts_10_x,ts%_10_x,efg%_10_x,3par_10_x,ftr_10_x,orb%_10_x,drb%_10_x,trb%_10_x,ast%_10_x,stl%_10_x,blk%_10_x,tov%_10_x,ortg_10_x,drtg_10_x,fg_max_10_x,fga_max_10_x,fg%_max_10_x,3p_max_10_x,3pa_max_10_x,3p%_max_10_x,ft_max_10_x,fta_max_10_x,ft%_max_10_x,orb_max_10_x,drb_max_10_x,trb_max_10_x,ast_max_10_x,stl_max_10_x,blk_max_10_x,tov_max_10_x,pf_max_10_x,pts_max_10_x,+/-_max_10_x,ts%_max_10_x,efg%_max_10_x,3par_max_10_x,ftr_max_10_x,orb%_max_10_x,drb%_max_10_x,trb%_max_10_x,ast%_max_10_x,stl%_max_10_x,blk%_max_10_x,tov%_max_10_x,usg%_max_10_x,ortg_max_10_x,drtg_max_10_x,total_10_x,home_10_x,fg_opp_10_x,fga_opp_10_x,fg%_opp_10_x,3p_opp_10_x,3pa_opp_10_x,3p%_opp_10_x,ft_opp_10_x,fta_opp_10_x,ft%_opp_10_x,orb_opp_10_x,drb_opp_10_x,trb_opp_10_x,ast_opp_10_x,stl_opp_10_x,blk_opp_10_x,tov_opp_10_x,pf_opp_10_x,pts_opp_10_x,ts%_opp_10_x,efg%_opp_10_x,3par_opp_10_x,ftr_opp_10_x,orb%_opp_10_x,drb%_opp_10_x,trb%_opp_10_x,ast%_opp_10_x,stl%_opp_10_x,blk%_opp_10_x,tov%_opp_10_x,ortg_opp_10_x,drtg_opp_10_x,fg_max_opp_10_x,fga_max_opp_10_x,fg%_max_opp_10_x,3p_max_opp_10_x,3pa_max_opp_10_x,3p%_max_opp_10_x,ft_max_opp_10_x,fta_max_opp_10_x,ft%_max_opp_10_x,orb_max_opp_10_x,drb_max_opp_10_x,trb_max_opp_10_x,ast_max_opp_10_x,stl_max_opp_10_x,blk_max_opp_10_x,tov_max_opp_10_x,pf_max_opp_10_x,pts_max_opp_10_x,+/-_max_opp_10_x,ts%_max_opp_10_x,efg%_max_opp_10_x,3par_max_opp_10_x,ftr_max_opp_10_x,orb%_max_opp_10_x,drb%_max_opp_10_x,trb%_max_opp_10_x,ast%_max_opp_10_x,stl%_max_opp_10_x,blk%_max_opp_10_x,tov%_max_opp_10_x,usg%_max_opp_10_x,ortg_max_opp_10_x,drtg_max_opp_10_x,total_opp_10_x,home_opp_10_x,won_10_x,season_10_x,home_next,team_opp_next_x,date_next,fg_10_y,fga_10_y,fg%_10_y,3p_10_y,3pa_10_y,3p%_10_y,ft_10_y,fta_10_y,ft%_10_y,orb_10_y,drb_10_y,trb_10_y,ast_10_y,stl_10_y,blk_10_y,tov_10_y,pf_10_y,pts_10_y,ts%_10_y,efg%_10_y,3par_10_y,ftr_10_y,orb%_10_y,drb%_10_y,trb%_10_y,ast%_10_y,stl%_10_y,blk%_10_y,tov%_10_y,ortg_10_y,drtg_10_y,fg_max_10_y,fga_max_10_y,fg%_max_10_y,3p_max_10_y,3pa_max_10_y,3p%_max_10_y,ft_max_10_y,fta_max_10_y,ft%_max_10_y,orb_max_10_y,drb_max_10_y,trb_max_10_y,ast_max_10_y,stl_max_10_y,blk_max_10_y,tov_max_10_y,pf_max_10_y,pts_max_10_y,+/-_max_10_y,ts%_max_10_y,efg%_max_10_y,3par_max_10_y,ftr_max_10_y,orb%_max_10_y,drb%_max_10_y,trb%_max_10_y,ast%_max_10_y,stl%_max_10_y,blk%_max_10_y,tov%_max_10_y,usg%_max_10_y,ortg_max_10_y,drtg_max_10_y,total_10_y,home_10_y,fg_opp_10_y,fga_opp_10_y,fg%_opp_10_y,3p_opp_10_y,3pa_opp_10_y,3p%_opp_10_y,ft_opp_10_y,fta_opp_10_y,ft%_opp_10_y,orb_opp_10_y,drb_opp_10_y,trb_opp_10_y,ast_opp_10_y,stl_opp_10_y,blk_opp_10_y,tov_opp_10_y,pf_opp_10_y,pts_opp_10_y,ts%_opp_10_y,efg%_opp_10_y,3par_opp_10_y,ftr_opp_10_y,orb%_opp_10_y,drb%_opp_10_y,trb

In [37]:
removed_columns = list(full.columns[full.dtypes == 'object']) + removed_columns
removed_columns

['team_x',
 'team_opp',
 'home_next',
 'team_opp_next_x',
 'date_next',
 'team_opp_next_y',
 'team_y',
 'season',
 'date',
 'won',
 'target',
 'team',
 'team_opp']

In [38]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [41]:
sfs.fit(full[selected_columns], full['target'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=40)

In [42]:
predictors= list(selected_columns[sfs.get_support()])
predictors

['3p%',
 'orb%',
 'tov%',
 'fg%_max',
 'ft%_max',
 'trb_max',
 'trb%_max',
 'drb%_opp',
 'stl%_opp',
 'blk%_opp',
 'ft%_max_opp',
 'trb_max_opp',
 'stl_max_opp',
 'blk%_max_opp',
 'usg%_max_opp',
 'fga_10_x',
 'blk_10_x',
 'pf_10_x',
 'stl%_10_x',
 'blk%_10_x',
 '+/-_max_10_x',
 'stl%_max_10_x',
 'fg_opp_10_x',
 'fta_opp_10_x',
 'ft%_opp_10_x',
 'pf_opp_10_x',
 'pts_opp_10_x',
 'drb_max_opp_10_x',
 'stl_max_opp_10_x',
 '3par_max_opp_10_x',
 'won_10_x',
 'pts_max_10_y',
 '+/-_max_10_y',
 'tov%_max_10_y',
 'fta_opp_10_y',
 'ast_opp_10_y',
 'blk_opp_10_y',
 'pf_opp_10_y',
 'blk%_opp_10_y',
 'blk%_max_opp_10_y']

In [61]:
predictions = backtest(full, rr, predictors)
predictions

,actual,prediction
4671,1,1
4672,1,0
4673,1,1
4674,1,1
4675,1,1
...,...,...
17739,2,0
17740,2,1
17741,2,0
17742,2,0


In [44]:
accuracy_score(predictions['actual'], predictions['prediction'])

0.620821540579821

In [62]:
full[['actual', 'prediction']] = predictions

In [63]:
summary_df_rr = full[['team_x', 'team_opp_next_x', 'date_next',  'prediction']]
summary_df_rr.tail(14)

,team_x,team_opp_next_x,date_next,prediction
17730,NYK,HOU,2023-03-27,1.0
17731,DET,MIL,2023-03-27,0.0
17732,PHI,DEN,2023-03-27,1.0
17733,LAC,CHI,2023-03-27,0.0
17734,DEN,PHI,2023-03-27,0.0
17735,MIL,DET,2023-03-27,1.0
17736,IND,DAL,2023-03-27,0.0
17737,NOP,POR,2023-03-27,1.0
17738,POR,NOP,2023-03-27,0.0
17739,CHI,LAC,2023-03-27,0.0


In [59]:
summary_df_xgb = full[['team_x', 'team_opp_next_x', 'date_next',  'prediction']]
summary_df_xgb.tail(14)

,team_x,team_opp_next_x,date_next,prediction
17730,NYK,HOU,2023-03-27,0.551698
17731,DET,MIL,2023-03-27,0.264057
17732,PHI,DEN,2023-03-27,0.610989
17733,LAC,CHI,2023-03-27,0.445371
17734,DEN,PHI,2023-03-27,0.334223
17735,MIL,DET,2023-03-27,0.895425
17736,IND,DAL,2023-03-27,0.388284
17737,NOP,POR,2023-03-27,0.605122
17738,POR,NOP,2023-03-27,0.183051
17739,CHI,LAC,2023-03-27,0.514275


In [64]:
summary_full = summary_df_rr
summary_full['percentage'] = summary_df_xgb['prediction']
summary_full.tail(14)

D:\Anaconda\envs\mlev\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,team_x,team_opp_next_x,date_next,prediction,percentage
17730,NYK,HOU,2023-03-27,1.0,0.551698
17731,DET,MIL,2023-03-27,0.0,0.264057
17732,PHI,DEN,2023-03-27,1.0,0.610989
17733,LAC,CHI,2023-03-27,0.0,0.445371
17734,DEN,PHI,2023-03-27,0.0,0.334223
17735,MIL,DET,2023-03-27,1.0,0.895425
17736,IND,DAL,2023-03-27,0.0,0.388284
17737,NOP,POR,2023-03-27,1.0,0.605122
17738,POR,NOP,2023-03-27,0.0,0.183051
17739,CHI,LAC,2023-03-27,0.0,0.514275
